In [144]:
import time
import numpy as np
from matplotlib import pyplot as plt
from keras.utils import np_utils
import keras.callbacks as cb
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import RMSprop, SGD, Adam, Nadam
from keras.datasets import mnist
from keras.layers import Merge
from keras.regularizers import l1, l2, activity_l2
from keras.layers.convolutional import Convolution1D, MaxPooling1D
import random
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
import scipy.stats as stats
import seaborn as sea
import plotly.plotly as py

In [145]:
metrics = pd.read_excel('GJR_design_metrics_161227.xls',header=0)
data_out = pd.read_excel('GJR_data_simplified_161227.xls',header=0,parse_cols="A,P")


In [146]:
metrics

,AlaCount,SCORE:,abd50_mean,abd50_min,abego_res_profile,abego_res_profile_penalty,avg_all_frags,avg_best_frag,bb,buried_minus_exposed,...,ssc50_min,sum_best_frags,total_score,tryp_cut_sites,two_core_each,unsat_hbond,unsat_hbond2,worst6frags,worstfrag,yhh_planarity
0,5,SCORE:,0.562854,0.252,0.366986,-0.033633,1.858251,0.446736,-0.422,3120.779,...,-3.442,14.7423,-132.050,7,0.200,2,2,5.8800,1.4102,0.021
1,3,SCORE:,0.501488,0.197,0.310619,-0.067378,1.616273,0.388597,-0.396,3574.936,...,-3.592,12.8237,-126.266,8,0.600,1,0,4.4051,1.0108,0.004
2,5,SCORE:,0.565878,0.378,0.300949,-0.036494,1.682794,0.340376,-0.370,3392.827,...,-3.146,11.2324,-129.425,7,0.400,1,0,3.7392,0.8318,0.382
3,4,SCORE:,0.630659,0.223,0.262549,-0.040904,1.751651,0.353094,-0.419,3245.684,...,-3.136,11.6521,-130.277,8,0.000,1,2,4.3599,1.0135,0.321
4,4,SCORE:,0.427780,0.122,0.313464,-0.049119,1.865065,0.415155,-0.312,2836.668,...,-3.613,13.7001,-115.069,8,0.200,2,0,6.0469,1.6184,0.000
5,4,SCORE:,0.568244,0.335,0.276435,-0.043818,1.692039,0.292764,-0.379,3497.616,...,-3.376,9.6612,-137.809,8,0.400,1,0,2.8477,0.7308,0.259
6,3,SCORE:,0.590463,0.409,0.231216,-0.075232,1.943675,0.394542,-0.371,3112.842,...,-2.541,13.0199,-140.892,8,0.200,1,1,4.5559,1.1970,0.113
7,4,SCORE:,0.503220,0.293,0.257976,-0.064685,1.838615,0.295482,-0.402,3592.006,...,-3.315,9.7509,-129.727,9,0.400,5,3,3.3682,0.7027,0.041
8,4,SCORE:,0.658098,0.458,0.287700,-0.048005,1.963658,0.364685,-0.382,3314.246,...,-2.780,12.0346,-129.966,6,0.600,1,1,4.6358,1.0525,0.006
9,5,SCORE:,0.496683,0.124,0.267917,-0.055935,1.890285,0.380809,-0.380,3176.020,...,-3.637,12.5667,-127.837,8,0.200,2,3,4.3329,1.1282,0.109


In [147]:
data_out.head()

,name,stability
0,EEHEE_0001.pdb,0.057998
1,EEHEE_0001.pdb_hp,-0.559243
2,EEHEE_0001.pdb_random,0.552774
3,EEHEE_0002.pdb,-0.232476
4,EEHEE_0002.pdb_hp,-0.380805


In [148]:
metrics['oldname'] = metrics['name']

In [149]:
data_out['name'] = data_out['name'].map(lambda x: str(x))
metrics['name'] = metrics['oldname'].map(lambda x: str(x)[:-10]+".pdb")

In [150]:
met_colnames = metrics.columns.tolist()
met_colnames.remove(u'name')
met_colnames.insert(0, u'name')
metrics = metrics[met_colnames]
metrics.head()

,name,AlaCount,SCORE:,abd50_mean,abd50_min,abego_res_profile,abego_res_profile_penalty,avg_all_frags,avg_best_frag,bb,...,sum_best_frags,total_score,tryp_cut_sites,two_core_each,unsat_hbond,unsat_hbond2,worst6frags,worstfrag,yhh_planarity,oldname
0,EEHEE_0001.pdb,5,SCORE:,0.562854,0.252,0.366986,-0.033633,1.858251,0.446736,-0.422,...,14.7423,-132.050,7,0.2,2,2,5.8800,1.4102,0.021,"EEHEE_0001_0001.pdb"""
1,EEHEE_0002.pdb,3,SCORE:,0.501488,0.197,0.310619,-0.067378,1.616273,0.388597,-0.396,...,12.8237,-126.266,8,0.6,1,0,4.4051,1.0108,0.004,"EEHEE_0002_0001.pdb"""
2,EEHEE_0003.pdb,5,SCORE:,0.565878,0.378,0.300949,-0.036494,1.682794,0.340376,-0.370,...,11.2324,-129.425,7,0.4,1,0,3.7392,0.8318,0.382,"EEHEE_0003_0001.pdb"""
3,EEHEE_0004.pdb,4,SCORE:,0.630659,0.223,0.262549,-0.040904,1.751651,0.353094,-0.419,...,11.6521,-130.277,8,0.0,1,2,4.3599,1.0135,0.321,"EEHEE_0004_0001.pdb"""
4,EEHEE_0005.pdb,4,SCORE:,0.427780,0.122,0.313464,-0.049119,1.865065,0.415155,-0.312,...,13.7001,-115.069,8,0.2,2,0,6.0469,1.6184,0.000,"EEHEE_0005_0001.pdb"""


In [151]:
merged_left = pd.merge(left=metrics,right=data_out, how='inner', on='name')
merged_left.head()

,name,AlaCount,SCORE:,abd50_mean,abd50_min,abego_res_profile,abego_res_profile_penalty,avg_all_frags,avg_best_frag,bb,...,total_score,tryp_cut_sites,two_core_each,unsat_hbond,unsat_hbond2,worst6frags,worstfrag,yhh_planarity,oldname,stability
0,EEHEE_0001.pdb,5,SCORE:,0.562854,0.252,0.366986,-0.033633,1.858251,0.446736,-0.422,...,-132.050,7,0.2,2,2,5.8800,1.4102,0.021,"EEHEE_0001_0001.pdb""",0.057998
1,EEHEE_0002.pdb,3,SCORE:,0.501488,0.197,0.310619,-0.067378,1.616273,0.388597,-0.396,...,-126.266,8,0.6,1,0,4.4051,1.0108,0.004,"EEHEE_0002_0001.pdb""",-0.232476
2,EEHEE_0003.pdb,5,SCORE:,0.565878,0.378,0.300949,-0.036494,1.682794,0.340376,-0.370,...,-129.425,7,0.4,1,0,3.7392,0.8318,0.382,"EEHEE_0003_0001.pdb""",0.052325
3,EEHEE_0004.pdb,4,SCORE:,0.630659,0.223,0.262549,-0.040904,1.751651,0.353094,-0.419,...,-130.277,8,0.0,1,2,4.3599,1.0135,0.321,"EEHEE_0004_0001.pdb""",-0.304535
4,EEHEE_0005.pdb,4,SCORE:,0.427780,0.122,0.313464,-0.049119,1.865065,0.415155,-0.312,...,-115.069,8,0.2,2,0,6.0469,1.6184,0.000,"EEHEE_0005_0001.pdb""",-1.352068


In [156]:
new_merged = merged_left.drop(['SCORE:','abd50_mean','abd50_min',
                               'buried_np_AFILMVWY','buried_np_AFILMVWY_per_res','oldname'],axis=1)
# merged_left = merged_left.drop('abd50_mean',axis=1)
# merged_left = merged_left.drop('abd50_min',axis=1)
new_merged

,name,AlaCount,abego_res_profile,abego_res_profile_penalty,avg_all_frags,avg_best_frag,bb,buried_minus_exposed,buried_np,buried_np_per_res,...,sum_best_frags,total_score,tryp_cut_sites,two_core_each,unsat_hbond,unsat_hbond2,worst6frags,worstfrag,yhh_planarity,stability
0,EEHEE_0001.pdb,5,0.366986,-0.033633,1.858251,0.446736,-0.422,3120.779,4660.890,113.680244,...,14.7423,-132.050,7,0.200,2,2,5.8800,1.4102,0.021,0.057998
1,EEHEE_0002.pdb,3,0.310619,-0.067378,1.616273,0.388597,-0.396,3574.936,4897.468,119.450439,...,12.8237,-126.266,8,0.600,1,0,4.4051,1.0108,0.004,-0.232476
2,EEHEE_0003.pdb,5,0.300949,-0.036494,1.682794,0.340376,-0.370,3392.827,4757.914,116.046683,...,11.2324,-129.425,7,0.400,1,0,3.7392,0.8318,0.382,0.052325
3,EEHEE_0004.pdb,4,0.262549,-0.040904,1.751651,0.353094,-0.419,3245.684,4668.842,113.874195,...,11.6521,-130.277,8,0.000,1,2,4.3599,1.0135,0.321,-0.304535
4,EEHEE_0005.pdb,4,0.313464,-0.049119,1.865065,0.415155,-0.312,2836.668,4363.834,106.434976,...,13.7001,-115.069,8,0.200,2,0,6.0469,1.6184,0.000,-1.352068
5,EEHEE_0006.pdb,4,0.276435,-0.043818,1.692039,0.292764,-0.379,3497.616,4817.308,117.495317,...,9.6612,-137.809,8,0.400,1,0,2.8477,0.7308,0.259,0.276606
6,EEHEE_0007.pdb,3,0.231216,-0.075232,1.943675,0.394542,-0.371,3112.842,4624.921,112.802951,...,13.0199,-140.892,8,0.200,1,1,4.5559,1.1970,0.113,0.167836
7,EEHEE_0008.pdb,4,0.257976,-0.064685,1.838615,0.295482,-0.402,3592.006,4962.003,121.024463,...,9.7509,-129.727,9,0.400,5,3,3.3682,0.7027,0.041,0.162208
8,EEHEE_0009.pdb,4,0.287700,-0.048005,1.963658,0.364685,-0.382,3314.246,4856.123,118.442024,...,12.0346,-129.966,6,0.600,1,1,4.6358,1.0525,0.006,-0.638625
9,EEHEE_0010.pdb,5,0.267917,-0.055935,1.890285,0.380809,-0.380,3176.020,4703.510,114.719756,...,12.5667,-127.837,8,0.200,2,3,4.3329,1.1282,0.109,-0.123946
